# K-Means Scratch

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import random
import numpy as np

## 1. Class Member & Cluster 

In [4]:
# Member store info of data points: tf_idf, news group, file name of text d
class Member:
    def __init__(self, r_d, label = None, doc_id = None):
        self._r_d = r_d
        self._label = label
        self._doc_id = doc_id

In [3]:
# Cluster: store data point
class Cluster:
    def __init__(self):
        # The underscore prefix is meant as a hint that a variable or method starting w "_" is intended for internal use
        # https://bit.ly/3pIEL8P
        self._centroid = None 
        self._members = []
    # Reset members (data points) in list of centroid
    def reset_members(self):
        self._members = []
    # Set centroid
    def set_centroid(self, new_centroid):
        self._centroid = new_centroid
    # Add new member (data point)
    def add_members(self, new_member):
        self._members.append(new_member)

## 2. Class KMeans

In [7]:
class KMeans:
    # Init with num_cluster, create K list store member of K centroid
    def __init__(self, num_clusters):
        self._num_clusters = num_clusters
        self._clusters = [Cluster() for i in range(self._num_clusters)]
        self._E = []       # List of centroids
        self._S = 0        # Overall similarity
    
    # Load Data
    def load_data(self, path = "/Users/charles/MLGT/SS2/Data/"):
        # Get data from each line with label, doc_id, index & tfidf of its vocab
        def sparse_to_dense(sparse_r_d, vocab_size):
            # Init list size vocal size to store vocal 
            r_d = [0.0 for _ in range(vocab_size)]
            
            # Split space & : in context data of each line 
            # Get index (id vocal of each line) & tfidfs
            indices_and_tfidfs = sparse_r_d.split()
            for index_and_tfidf in indices_and_tfidfs:
                index = int(index_and_tfidf.split(':')[0])
                tfidf = float(index_and_tfidf.split(':')[1])
                r_d[index] = tfidf
            return np.array(r_d)    
                
        # Open file (newsgroup, id, context)
        with open(path + "data_tf_idf.txt") as f:
            data_lines = f.read().splitlines()
        # Get size file vocal TF-IDF
        with open(path + "words_idfs.txt") as f:
            vocab_size = len(f.read().splitlines())

        # Member store info of data points: tf_idf, news group, file name of text d
        self._data = []
        # Count number of label (newsgroup)
        self._label_count = defaultdict(int)
        # Iterating sequence of pairs with counter
        for data_id, d in enumerate(data_lines):
            features = d.split('<fff>')
            label, doc_id = int(features[0]), int(features[1])
            self._label_count[label] += 1
            r_d = sparse_to_dense(sparse_r_d=features[2], vocab_size=vocab_size)

            # Append data with class Member
            self._data.append(Member(r_d=r_d, label=label, doc_id=doc_id))
    
    # Prepare for RUN Function
    # Random element to init centroid 
    def random_init(self, seed_value):
        random.seed(seed_value)
        # Crawl list members
        members = [member._r_d for member in self._data]
        # random.choice: random sample from 1-D array
        # Same as np.arrange without same num in array
        pos = np.random.choice(len(self._data), self._num_clusters, replace=False)
        centroid = []
        for i in pos:
            centroid.append(members[i])
        # Update centroid
        self._E = centroid
        for i in range(self._num_clusters):
            self._clusters[i].set_centroid(centroid[i])

    # Calculate similarity from centroid to another member
    # https://www.machinelearningplus.com/nlp/cosine-similarity/
    def compute_similarity(self, member, centroid):
        # K(X, Y) = <X, Y> / (||X||*||Y||)
        return cosine_similarity([member._r_d], [centroid])

    # Assign member to cluster
    def select_cluster_for(self, member):
        best_fit_cluster = None
        # cos = -1 <=> 180
        max_similarity = -1
        # Check all list member cluster 
        for cluster in self._clusters:
            # Check similarity for each member with centroid of each cluster
            similarity = self.compute_similarity(member, cluster._centroid)
            # Check to peak max similarity LOCAL
            if similarity > max_similarity:
                best_fit_cluster = cluster
                max_similarity = similarity
        best_fit_cluster.add_members(member)
        return max_similarity

    # Recompute centroid 
    def update_centroid_of(self, cluster):
        # List all TF-IDF
        member_r_ds = [member._r_d for member in cluster._members]
        avg_r_d = np.mean(member_r_ds, axis=0)
        sqrt_sum_sqr = np.sqrt(np.sum(avg_r_d ** 2))
        new_centroid = avg_r_d / sqrt_sum_sqr

        # Update centroid
        cluster._centroid = new_centroid

    # Stopping Condition
    def stopping_condition(self, criterion, threshold):
        criteria = ['centroid', 'similarity', 'max_iters']
        assert criterion in criteria
        if criterion == 'max_iters':
            if self._iteration >= threshold:
                return True
            else:
                return False
        elif criterion == 'centroid':
            E_new = [list(cluster._centroid) for cluster in self._clusters]
            E_new_minus_E = [centroid for centroid in E_new if centroid not in self._E]
            self._E = E_new
            if len(E_new_minus_E) <= threshold:
                return True
            else:
                return False
        else:
            new_S_minus_S = self._new_S - self._S
            self._S = self._new_S
            if new_S_minus_S <= threshold:
                return True
            else:
                return False    

    # Loop update cluster until convergence 
    # Step 1: For each instance, assign it to nearest centroid
    # Step 2: For each cluster, recompute its centroid
    def run(self, seed_value, criterion, threshold):
        self.random_init(seed_value)
        self._iteration = 0
        while True:
            # Reset clusters, retain only centroids
            for cluster in self._clusters:
                cluster.reset_members()
                self._new_S = 0
            # Select nearest member for each cluster
            for member in self._data:
                max_s = self.select_cluster_for(member)
                self._new_S += max_s
            # Recompute centroid 
            for cluster in self._clusters:
                self.update_centroid_of(cluster)

            # Stop loop with criterion
            self._iteration += 1
            if self.stopping_condition(criterion, threshold):
                break

    # Evaluation
    # Purity
    def compute_purity(self):
        majority_sum = 0
        for cluster in self._clusters:
            member_labels = [member._label for member in cluster._members]
            max_count = max([member_labels.count(label) for label in range(20)])
            majority_sum += max_count
        return majority_sum * 1. / len(self._data)

    # NMI
    def compute_NMI(self):
        I_value, H_omega, H_C, N = 0., 0., 0., len(self._data)
        for cluster in self._clusters:
            wk = len(cluster._members) * 1. 
            H_omega += - wk / N * np.log10(wk / N)
            member_labels = [member._label for member in cluster._members]
        for label in range(20):
            wk_cj = member_labels.count(label) * 1.
            cj = self._label_count[label]
            I_value += wk_cj / N * np.log10(N * wk_cj / (wk * cj) + 1e-12)
        for label in range(20):
            cj = self._label_count[label] * 1.
            H_C += - cj / N * np.log10(cj / N)
        return I_value * 2. / (H_omega + H_C)

## 3. Evaluation

In [84]:
kmeans = KMeans(num_clusters=8)
kmeans.load_data()
kmeans.run(seed_value=42, criterion='similarity', threshold=1e-3)
print(kmeans.compute_purity(), kmeans.compute_NMI())

0.2644593017085854 0.05430282948565754
